In [40]:
import pickle
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from time import time
from ultralytics import YOLO
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
#from dgl.nn.pytorch.conv import RelGraphConv
from dgl.nn import GraphConv
#import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import cv2
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from keras.applications.inception_v3 import preprocess_input

global_resize_image=(512,512)

In [3]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text


In [4]:
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# load training dataset (6K)
#filename = 'Flickr_1k.trainImages.txt'
filename='D:\\Paper\\Mimic Human Level Intelligence in Image Descriptioning\\train_split.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))

Dataset: 14575


In [5]:
images = 'D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K/'

# Create a list of all image names in the directory
img = glob.glob(images + '*.jpg')

In [69]:
print(images)

D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K/


In [6]:
train_images_file = 'D:\\Paper\\Mimic Human Level Intelligence in Image Descriptioning\\train_split.txt'
# Read the train image names in a set
train_images = set(open(train_images_file, 'r').read().strip().split('\n'))

# Create a list of all the training images with their full path names
train_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in train_images: # Check if the image belongs to training set
        train_img.append(i) # Add it to the list of train images

In [7]:
# Below file conatains the names of images to be used in test data
test_images_file = 'D:\\Paper\\Mimic Human Level Intelligence in Image Descriptioning\\test_split.txt'
# Read the validation image names in a set# Read the test image names in a set
test_images = set(open(test_images_file, 'r').read().strip().split('\n'))

# Create a list of all the test images with their full path names
test_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in test_images: # Check if the image belongs to test set
        test_img.append(i) # Add it to the list of test images

In [9]:
len(test_img)

2489

In [10]:
def preprocess(image_path):
    
    img = image.load_img(image_path, target_size=(299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [11]:
model = InceptionV3(weights='imagenet')
model_new = Model(model.input, model.layers[-2].output)

96124928/96112376 [==============================] - 10s 0us/step


In [ ]:
# Convert to DataFrame
def Convert_to_DataFrame(results):
    boxes = results[0].boxes.xyxy.tolist()
    classes = results[0].boxes.cls.tolist()
    names = results[0].names
    confidences = results[0].boxes.conf.tolist()
    data = {'xmin': [box[0] for box in boxes],
        'ymin': [box[1] for box in boxes],
        'xmax': [box[2] for box in boxes],
        'ymax': [box[3] for box in boxes],
        'confidence': confidences,
        'class': classes,
        'name': [names[int(cls)] for cls in classes]}
    output_df = pd.DataFrame(data)
    #print(output_df)
    return output_df

In [14]:
#Object Detection using YOLOv8
def detect_objects(imagePath):
    image=cv2.imread(imagePath)
    image=cv2.resize(image,global_resize_image)
    model = YOLO('yolov8x.pt')
    results=model(image)
    output_df=Convert_to_DataFrame(results)
    return output_df 

In [13]:
def extract_nodes(image):
    output_df=detect_objects(image)
    image_nodes=[]
    for i in range(output_df.shape[0]):
        #print(output_df.iloc[i]['name'])
        data={"object_id":i,
        "start_point":(round(output_df.iloc[i]['xmin']),round(output_df.iloc[i]['ymin'])),
        "ending_point":(round(output_df.iloc[i]['xmax']),round(output_df.iloc[i]['ymax'])),
        "label":output_df.iloc[i]['name']
        }
        image_nodes.append(data)
    return image_nodes  

In [74]:
def generate_nodeImages(image,nodes):
    image=cv2.resize(image,(256,256))
    #image = cv2.rectangle(image, (106,19), (188,133), (255, 0, 0) , 2) 
    #crop=image[19:133,106:188]
    #cv2.imshow('image',crop)
    #cv2.waitKey()
    segment_array=[]
    for i in range(len(nodes)):
        start=nodes[i].get('start_point')
        end=nodes[i].get('ending_point')
        #print(end[1])
        a,b,c,d=start[1],end[1],start[0],end[0]
        crop=image[a:b,c:d]
        segment_array.append(crop)
    segment_array.append(image)    
    return segment_array    

In [36]:
def featuresVGG(segment_array):
    VGG_features=[]
    for image in segment_array:
        image=cv2.resize(image,(299,299))
        image=img_to_array(image)
        image=image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
        image=preprocess_input(image)
        feature=model.predict(image)
        VGG_features.append(feature)
    return VGG_features    

In [19]:
image=cv2.imread(r"D:\\Paper\\Mimic Human Level Intelligence in Image Descriptioning\\Flicker8k_Dataset\\109202801_c6381eef15.jpg")

In [20]:
nodes=extract_nodes(image)

Using cache found in C:\Users\Riddhick/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-2-22 Python-3.9.5 torch-2.2.0+cpu CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 29.9MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [21]:
len(nodes)

3

In [22]:
segment_array=generate_nodeImages(image,nodes)

In [37]:
VGG=featuresVGG(segment_array)

In [28]:
len(segment_array)

4

In [31]:
VGG[2].shape

(1, 1000)

In [49]:
def graph_features(feature_list):
    
    g = dgl.DGLGraph()
    num_nodes = len(feature_list)
    g.add_nodes(num_nodes)
    features = torch.tensor(feature_list, dtype=torch.float32)
    g.ndata['features'] = features
    num_edges = num_nodes*num_nodes-1 
    g.add_edges(torch.randint(num_nodes, (num_edges,)), torch.randint(num_nodes, (num_edges,)))
    g.edata['edge_weights'] = nn.Parameter(torch.rand(num_edges, requires_grad=True))

# Define a Graph Convolutional Network (GCN) model
    class GCN(nn.Module):
        def __init__(self, in_feats, hidden_feats):
            super(GCN, self).__init__()
            self.conv = GraphConv(in_feats, hidden_feats)

        def forward(self, g, features):
            # Perform graph convolution with learnable edge weights
            h = self.conv(g, features)
            h = F.relu(h)
            return h

# Reshape the features tensor
    features = features.squeeze(1)

    # Instantiate the GCN model
    in_feats = features.shape[1] # Number of input features
    hidden_feats = 256 # Number of hidden units
    model = GCN(in_feats, hidden_feats)

    # Forward pass
    output_gcn = model(g, features)
    output=output_gcn.detach().numpy()
    output=output.transpose()
    #b=output.reshape(1,-1)
    return output

In [50]:
x=graph_features(VGG)

In [51]:
x.shape

(256, 4)

In [57]:
def feature_reduction(features):
    pca = PCA(n_components=1)
    pca.fit(features)
    s=StandardScaler()
    x=pca.transform(features)
    x=np.reshape(x,x.shape[0])
    return x

In [59]:
y=feature_reduction(x)

In [60]:
y.shape

(256,)

In [79]:
def complete_extraction(image_path):
    img= cv2.imread(image_path)
    nodes=extract_nodes(img)
    segment_array=generate_nodeImages(img,nodes)
    feature_vectors=featuresVGG(segment_array)
    print(len(nodes))
    gcn_features=graph_features(feature_vectors)
    feature=feature_reduction(gcn_features)

In [87]:
m=complete_extraction("D:\\Paper\\Mimic Human Level Intelligence in Image Descriptioning\\visual_genome\\images\\VG_100K\\112.jpg")

Using cache found in C:\Users\Riddhick/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-2-22 Python-3.9.5 torch-2.2.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


10


In [85]:
start = time()
encoding_train = {}
for i in range(10):
    print(train_img[i])
print("Time taken in seconds =", time()-start)

D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\1001.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\1025.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\1060.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\1068.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\108.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\1081.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\1082.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\1090.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\1122.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/image